In [1]:
'''
This program is a full start to end implementation of a CNN that classifies
candlestick charts based on the price being higher or lower after a certain
period of time.

The program that does the following:

1. Calculate price increase or decrease after period
2. Create and resize candlestick images
3. Set up convolutional neural network (CNN)
4. Train CNN on training data
5. Test CNN on test data

Future Tasks:
- Calculate PnL values based on stop loss and take profit
- Better understanding of CNN and definition
- Better understanding of CNN training and testing
'''

'\nThis program is a full start to end implementation of a CNN that classifies\ncandlestick charts based on the price being higher or lower after a certain\nperiod of time.\n\nThe program that does the following:\n\n1. Calculate price increase or decrease after period\n2. Create and resize candlestick images\n3. Set up convolutional neural network (CNN)\n4. Train CNN on training data\n5. Test CNN on test data\n\nFuture Tasks:\n- Calculate PnL values based on stop loss and take profit\n- Better understanding of CNN and definition\n- Better understanding of CNN training and testing\n'

In [2]:
# Configurable Parameters

ticker = "AAPL"
start_date = '2024-01-01'
end_date = '2024-02-01'
timeframe = '1h'
length = 8
save_images = True

# lookback = 30
# deadline = 30

# take_profit = 15
# stop_loss = 15

In [3]:
# Other Parameters

charts_path = 'charts/'
chart_img_name = 'chart_'
chart_img_ext = '.jpg'

In [4]:
# Library and Tool Imports

import csv
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import os 
import shutil
import yfinance as yf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [5]:
# Download Stock Data

data = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

[*********************100%%**********************]  1 of 1 completed


In [6]:
# Calculate Increase and Decrease Values

closing_prices = data['Close']
pnl = []

for i in range(len(closing_prices) - length):
    pnl.append(int(closing_prices.iloc[i + length] > closing_prices.iloc[i]))
    

In [7]:
# Create and Save Image Charts

if os.path.exists(charts_path):
    shutil.rmtree(charts_path)

os.mkdir(charts_path)

for i in range(len(data) - length):
    mpf.plot(
        data.iloc[i:i+length],
        type='candle',
        ylabel='',
        figscale=0.5,
        datetime_format='',
        tight_layout=True,
        savefig=f'{charts_path}{chart_img_name}{i}{chart_img_ext}')

In [8]:
# Resize Image Charts

image_width, image_height = Image.open(f'{charts_path}{chart_img_name}{0}{chart_img_ext}').size

image_width, image_height = image_width // 10 * 10, image_height // 10 * 10

for i in range(len(pnl)):
    path = f'{charts_path}{chart_img_name}{i}{chart_img_ext}'
    with Image.open(path) as img:
        img = img.resize((image_height, image_width))
        img.save(path)

In [9]:
# Create Arrays from Image Data

images = []

for i in range(len(pnl)):
    img = plt.imread(f'{charts_path}{chart_img_name}{i}{chart_img_ext}')
    images.append(img / 255.)

if not save_images:
    shutil.rmtree(charts_path)

In [10]:
# Split Data into Training and Testing Sets

images = np.array(images)
labels = np.array(pnl)

X_train, X_test, y_train, y_test = images[:100], images[100:], labels[:100], labels[100:]

# X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [11]:
# Build the CNN Model

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/Users/cameron.beneteau/CompVision-W2024/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 388, 258, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 194, 129, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 192, 127, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 96, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 94, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 47, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 180480)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,101,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,194,945 (88.48 MB)

 Trainable params: 23,194,945 (88.48 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=25
)

Epoch 1/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5526 - loss: 0.6907 - val_accuracy: 0.1538 - val_loss: 0.7101
Epoch 2/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5919 - loss: 0.6892 - val_accuracy: 0.1282 - val_loss: 0.7179
Epoch 3/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5141 - loss: 0.6922 - val_accuracy: 0.1026 - val_loss: 0.7217
Epoch 4/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 861ms/step - accuracy: 0.5785 - loss: 0.6855 - val_accuracy: 0.1026 - val_loss: 0.7283
Epoch 5/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 916ms/step - accuracy: 0.5906 - loss: 0.6888 - val_accuracy: 0.1026 - val_loss: 0.7320
Epoch 6/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 882ms/step - accuracy: 0.5533 - loss: 0.6882 - val_accuracy: 0.1026 - val_loss: 0.7317
Epoch 7/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.4884 - loss: 0.7041 - val_accuracy: 0.1026 - val_loss: 0.7274
Epoch 8/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 838ms/step - accuracy: 0.5873 - loss: 0.6872 - val_accuracy: 0.1282 - val_loss: 0.7250
Epoc

In [24]:
# Model Evaluation with Testing Data

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy*100}%")
print(f"Test loss: {loss*100}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.1100 - loss: 0.7314
Test accuracy: 10.256410390138626%
Test loss: 73.37256073951721%


In [14]:
# Save Training and Test Results

results_file = "results.csv"

file = open(results_file, 'a', newline='')

writer = csv.writer(file, delimiter=',')
writer.writerow([ticker, start_date, end_date, timeframe, length, round(loss, 5), round(accuracy, 5)])

file.close()

In [15]:
'''
Meeting Notes
- Forward bias (look back instead of forward)
- Last candle instead of first
- Testing and training data needs to be split chronologically

To-Do
- Fix up PnL calculation
- Setup up testing framework to use PnL, images, and others
- Dynamic testing framework (lookback, deadline, take profit, stop loss, etc.) 
'''

'\nMeeting Notes\n- Forward bias (look back instead of forward)\n- Last candle instead of first\n- Testing and training data needs to be split chronologically\n\nTo-Do\n- Fix up PnL calculation\n- Setup up testing framework to use PnL, images, and others\n- Dynamic testing framework (lookback, deadline, take profit, stop loss, etc.) \n'